In [67]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from pyproj import Transformer
from shapely import Point, wkt
import pandas as pd
import os

In [3]:
# Load the shapefiles
dams_csv = pd.read_csv(r"D:\CS\GitHub\Nuclear-Site-Selection\dam\dam.csv")
dams = gpd.read_file(r"D:\CS\GitHub\Nuclear-Site-Selection\dam\Dam.shp")
airports = gpd.read_file(r'D:\CS\GitHub\Nuclear-Site-Selection\Airport\Airport.shp')

In [45]:
airports.to_crs(epsg=4326)

,objectid_1,objectid,name,type,district,state,geometry
0,1,1,"Trivandrum International Airport, Thiruvananth...",INTERNATIONAL,Thiruvananthapuram,KL,POINT (76.92186 8.48731)
1,2,2,"Tuticorin Airport, Thoothukudi",DOMESTIC AIRPORT,Thoothukkudi,TN,POINT (78.02687 8.72607)
2,3,3,"Madurai Airport, Madurai",CUSTOMS,Madurai,TN,POINT (78.08942 9.83854)
3,4,4,"Cochin International Airport, Kochi",INTERNATIONAL,Ernakulam,KL,POINT (76.39381 10.15731)
4,5,5,"Thanjavur Airport, Thanjavur",DOMESTIC CIVIL ENCLAVES AIRPORTS,Thanjavur,TN,POINT (79.10007 10.72419)
...,...,...,...,...,...,...,...
146,147,147,"Atal Bihari Vajpayee International Airport, Si...",DOMESTIC AIRPORT,Deoghar,JH,POINT (86.70354 24.44689)
147,148,148,Dhalbhumgarh Airport,DOMESTIC AIRPORT,Purbi Singhbhum,JH,POINT (86.55762 22.51995)
148,149,149,"Adani Airport, Mundra Airport",DOMESTIC AIRPORT,Kachchh,GJ,POINT (69.76367 22.83574)
149,150,150,"AIZAWL (TURIAL), Aizawl",DOMESTIC AIRPORT,Aizawl,MZ,POINT (92.80233 23.74198)


In [60]:
gridpoints = pd.read_csv(r"D:\CS\GitHub\Nuclear-Site-Selection\gridpoints_csv_dams\Aad_Dam.csv")

In [64]:
def find_nearest_airport(point, airports):
    distances = airports.geometry.apply(lambda x: point.distance(x))
    nearest_index = distances.idxmin()
    nearest_airport = airports.loc[nearest_index]
    return nearest_airport['name'], distances[nearest_index]

In [62]:
#Iterate through gridpoints and find the nearest airport for each
for index, row in gridpoints.iterrows():
    point = wkt.loads(row['geometry'])  # Parse the geometry column
    nearest_airport_name, distance = find_nearest_airport(point, airports)
    distance_km = distance / 1000  # Convert to kilometers
    
    gridpoints.at[index, 'nearest_airport'] = nearest_airport_name
    gridpoints.at[index, 'distance_to_nearest_airport_km'] = distance_km

gridpoints = gridpoints.drop(columns=['Unnamed: 0.2','Unnamed: 0','index_righ','Shape_Leng','Shape_Area','Unnamed: 0.1','State_LGD'])

In [63]:
gridpoints

,dam_id,STATE,geometry,population,nearest_airport,distance_to_nearest_airport_km
0,Aad Dam,MAHARASHTRA,POINT (3333625.1089999974 3600497.4990000017),214.729874,"Nashik Airport, Ojhar",40.141438
1,Aad Dam,MAHARASHTRA,POINT (3333625.1089999974 3601497.4990000017),223.124924,"Nashik Airport, Ojhar",40.384344
2,Aad Dam,MAHARASHTRA,POINT (3333625.1089999974 3602497.4990000017),146.727386,"Nashik Airport, Ojhar",40.650405
3,Aad Dam,MAHARASHTRA,POINT (3333625.1089999974 3603497.4990000017),153.979294,"Nashik Airport, Ojhar",40.939169
4,Aad Dam,MAHARASHTRA,POINT (3333625.1089999974 3604497.4990000017),154.121307,"Nashik Airport, Ojhar",41.250161
...,...,...,...,...,...,...
242,Aad Dam,MAHARASHTRA,POINT (3351625.1089999974 3607497.4990000017),170.899353,"Nashik Airport, Ojhar",26.614136
243,Aad Dam,MAHARASHTRA,POINT (3352625.1089999974 3602497.4990000017),183.064301,"Nashik Airport, Ojhar",23.008767
244,Aad Dam,MAHARASHTRA,POINT (3352625.1089999974 3603497.4990000017),193.780762,"Nashik Airport, Ojhar",23.515176
245,Aad Dam,MAHARASHTRA,POINT (3352625.1089999974 3604497.4990000017),158.986542,"Nashik Airport, Ojhar",24.052520


In [93]:
base_dir = r"D:\CS\GitHub\Nuclear-Site-Selection\gridpoints_csv_dams"

def read_nearest_airport(dam_name):
    safe_dam_name = dam_name.replace(' ','_').replace('/','_').replace(':','').replace('\n','_').replace("?","")
    folder_path = os.path.join(base_dir,safe_dam_name)
    folder_path+='.csv'
    gridpoints = pd.read_csv(folder_path)
    gridpoints['nearest_airport'] = 0
    gridpoints['distance_to_nearest_airport_km'] = 0
    for index, row in gridpoints.iterrows():
        point = wkt.loads(row['geometry'])  # Parse the geometry column
        nearest_airport_name, distance = find_nearest_airport(point, airports)
        distance_km = distance / 1000  # Convert to kilometers
        
        gridpoints.at[index, 'nearest_airport'] = nearest_airport_name
        gridpoints.at[index, 'distance_to_nearest_airport_km'] = distance_km

    columns_to_drop = ['Unnamed: 0.2', 'Unnamed: 0', 'index_righ', 'Shape_Leng', 'Shape_Area', 'Unnamed: 0.1', 'State_LGD']
    gridpoints = gridpoints.drop(columns=[col for col in columns_to_drop if col in gridpoints.columns])


    gridpoints.to_csv(folder_path)

            

In [94]:
dams_csv['dm_name'].apply(lambda dam_name:read_nearest_airport(dam_name))

C:\Users\Nirman\AppData\Local\Temp\ipykernel_26088\1979460089.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Atal Bihari Vajpayee International Airport, Simra, Deoghar' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gridpoints.at[index, 'nearest_airport'] = nearest_airport_name
C:\Users\Nirman\AppData\Local\Temp\ipykernel_26088\1979460089.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '78.40313479649667' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gridpoints.at[index, 'distance_to_nearest_airport_km'] = distance_km
C:\Users\Nirman\AppData\Local\Temp\ipykernel_26088\1979460089.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Atal Bihari Vajpa

0       None
1       None
2       None
3       None
4       None
        ... 
5614    None
5615    None
5616    None
5617    None
5618    None
Name: dm_name, Length: 5619, dtype: object

In [77]:
gridpoints = pd.read_csv(r"D:\CS\GitHub\Nuclear-Site-Selection\gridpoints_csv_dams\Aad_Dam.csv")

In [78]:
gridpoints

,Unnamed: 0,dam_id,STATE,geometry,population,nearest_airport,distance_to_nearest_airport_km
0,0,Aad Dam,MAHARASHTRA,POINT (3333625.1089999974 3600497.4990000017),214.729874,"Nashik Airport, Ojhar",40.141438
1,1,Aad Dam,MAHARASHTRA,POINT (3333625.1089999974 3601497.4990000017),223.124924,"Nashik Airport, Ojhar",40.384344
2,2,Aad Dam,MAHARASHTRA,POINT (3333625.1089999974 3602497.4990000017),146.727386,"Nashik Airport, Ojhar",40.650405
3,3,Aad Dam,MAHARASHTRA,POINT (3333625.1089999974 3603497.4990000017),153.979294,"Nashik Airport, Ojhar",40.939169
4,4,Aad Dam,MAHARASHTRA,POINT (3333625.1089999974 3604497.4990000017),154.121307,"Nashik Airport, Ojhar",41.250161
...,...,...,...,...,...,...,...
242,242,Aad Dam,MAHARASHTRA,POINT (3351625.1089999974 3607497.4990000017),170.899353,"Nashik Airport, Ojhar",26.614136
243,243,Aad Dam,MAHARASHTRA,POINT (3352625.1089999974 3602497.4990000017),183.064301,"Nashik Airport, Ojhar",23.008767
244,244,Aad Dam,MAHARASHTRA,POINT (3352625.1089999974 3603497.4990000017),193.780762,"Nashik Airport, Ojhar",23.515176
245,245,Aad Dam,MAHARASHTRA,POINT (3352625.1089999974 3604497.4990000017),158.986542,"Nashik Airport, Ojhar",24.052520
